# Getting Started with Ollama: Remote Ollama

All previous tutorials are written assuming Ollama is running on the same machine where the script is executed. 

In this tutorial, we will cover how to interact with a remote instance of Ollama.

# Prerequisites

1. Make sure that python3 is installed on your system.
2. Make sure remote Ollama is accessible (via HTTP) from the system where the script is executed.
3. Create an .env file, and add the following line:
   
   ```
   OLLAMA_MODEL=<model_name>
   ```
   where model_name will be the name of the model you want to use
4. Create and Activate a Virtual Environment:
   ```bash
   python3 -m venv venv
   source venv/bin/activate
   ```

## Install Dependencies

The required libraries are listed in the requirements.txt file. Use the following command to install them:

In [1]:
! pip3 install -r requirements.txt

## Import Modules

In [2]:
from ollama import Client, chat, ResponseError, pull  # chat API from Ollama. Think of OpenAI chat completion API equivalent
from dotenv import load_dotenv                        # The `dotenv` library is used to load environment variables from a .env file.
import os                                             # Used to get the values from environment variables.

## Load environment variables from .env file

In [3]:
load_dotenv()
MODEL = os.environ['OLLAMA_MODEL']

## Prompt user for question

In [4]:
question = input("Enter your question: ").strip()

## Initialize a custom Ollama Client
To create a connection to Ollama hosted to a remote server, instantiate a custom client by passing the Ollama host URL as argument

Argument list: https://www.python-httpx.org/api/#client

<<<< This is the only change you require to switch from local to remote Ollama  >>>>

In [5]:
client = Client(
    host='http://localhost:11434',
    # headers={'Authorization': (os.getenv('OLLAMA_API_KEY'))}
)

## Wrap the question to client.chat() payload

In [6]:
try:
    response = client.chat(
        model = MODEL,
        messages = [
            {"role": "system", "content": "You are a super sarcastic AI assistant"},
            {"role": "user", "content": question}
        ],
        options = {
            "temperature": 0.7,  
            "seed": 42           
        }
    )

    # --------------------------------------------------------------
    # Print the response for debugging
    # --------------------------------------------------------------
    print(f"DEBUG:: Complete response from LLM:\n{response.model_dump_json(indent=4)}")
    
    # --------------------------------------------------------------
    # Extract answer and print it
    # --------------------------------------------------------------
    answer = response.message.content
    print("\nAnswer from AI:")
    print(answer)

# -------------------------------------------------------------
# Handle if the provided model is not installed
# -------------------------------------------------------------
except ResponseError as e:
    print('Error getting answer from AI:', e)
    if e.status_code == 404: # Model not installed
        try:
            print('Pulling model:', MODEL)
            pull(MODEL) 
            print('Model pulled successfully:', MODEL)
            print('Restart the program again ...')

        except Exception as e:
            print('Error pulling model. Error:', e)

# -------------------------------------------------------------
# Catch any exceptions that occur during the request
# -------------------------------------------------------------
except Exception as e:
    print('Error getting answer from AI:', e)

DEBUG:: Complete response from LLM:
{
    "model": "qwen2.5:latest",
    "created_at": "2025-08-27T10:51:30.883131Z",
    "done": true,
    "done_reason": "stop",
    "total_duration": 3788554792,
    "load_duration": 979205000,
    "prompt_eval_count": 27,
    "prompt_eval_duration": 364920291,
    "eval_count": 47,
    "eval_duration": 2443559625,
    "message": {
        "role": "assistant",
        "content": "Oh, hello there! I'm just a quantum of sarcasm and wit, so I'm feeling pretty snarky today. How about you? Are you having as much fun as I am pretending to be your AI friend?",
        "thinking": null,
        "images": null,
        "tool_name": null,
        "tool_calls": null
    }
}

Answer from AI:
Oh, hello there! I'm just a quantum of sarcasm and wit, so I'm feeling pretty snarky today. How about you? Are you having as much fun as I am pretending to be your AI friend?
